In [ ]:
#Importing Libraries we will need
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt

In [ ]:
#defining number of data points
NUM_SAMPLES = 200

#Simulating data. x1 is random from 0 to 1
x1 = np.random.rand(NUM_SAMPLES)
#x2 is very correlated to x1, with small random fluctuations
x2 = x1 + np.random.rand(NUM_SAMPLES)/10

#x3-5 are random
x3 = np.random.rand(NUM_SAMPLES)
x4 = np.random.rand(NUM_SAMPLES)
x5 = np.random.rand(NUM_SAMPLES)

#y is generated by x1 - x2/2 + noise
y = x1 - x2/2 + np.random.rand(NUM_SAMPLES)/10

#Lets pack it all into a data frame. A data frame is similar to a SQL table, displayed below
df = pd.DataFrame({'x1':x1, 'x2':x2,'x3':x3,'x4':x4,'x5':x5, 'y':y})

df.head()

In [ ]:
#X is the input features. Think of these like marketing spend by day
X = df[['x1', 'x2', 'x3', 'x4', 'x5']]
#Y is the target variable. Think of this like the sales by day, as a function of marketing spend
Y = df['y']

In [ ]:
#Lets visualize the correlations between x1 vs Y and x2 vs Y
plt.figure(figsize=(10,5))
plt.subplot(1,2,1).scatter(x1,Y)
plt.title('x1 vs Y\n correlation:' + str(np.corrcoef(x1, Y)[0,1].round(4)))
plt.subplot(1,2,2).scatter(x2,Y)
plt.title('x2 vs Y\n correlation:' + str(np.corrcoef(x2, Y)[0,1].round(4)))
plt.show()


In [ ]:
#Here we build a regression model using sklearn Ridge and fit it to our dataset
RegressionModel = Ridge(1e-3).fit(X,Y)

#The coefficients are the ROI of each input feature on our output target
coefs = RegressionModel.coef_.round(2)

In [ ]:
print('Regression coefficients from Sklearn')
print(list(zip(['x1', 'x2', 'x3', 'x4', 'x5'], coefs)))

In [ ]:
# Below we solve the same problem using tensorflow. Note the significant added complexity to perform the same task
# The reason for using tensorflow as opposed to sklearn will come up later, in that it gives us added control
import tensorflow as tf
tf.reset_default_graph()

# We define our inputs
x_ph = tf.placeholder(tf.float32, (None, 5))
y_ph = tf.placeholder(tf.float32, (None))

#We define a linear regression layer
yhat = tf.layers.dense(x_ph, 1, name='regression_coefficients')

#we calculate our loss
loss = tf.reduce_mean(tf.square(yhat[:,0] - y_ph))

#We minimize the loss
optimizer = tf.train.GradientDescentOptimizer(.1).minimize(loss)

#Tensorflow necessary initialization of our model.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#We take optimization steps training the model to fit X, Y
for i in range(4000):
    sess.run(optimizer, {x_ph:X, y_ph:Y})
    
#We analyze the regression coefficients (ROI) the model learned
with tf.variable_scope("regression_coefficients", reuse=True):
    weights = tf.get_variable("kernel")
print('Regression coefficients From Tensorflow:')

coefs = sess.run(weights[:,0]).round(2)
print(list(zip(['x1', 'x2', 'x3', 'x4', 'x5'], coefs)))

In [ ]:
# Below we solve the same problem using tensorflow. Note the significant added complexity to perform the same task
# The reason for using tensorflow as opposed to sklearn will come up later, in that it gives us added control
import tensorflow as tf
tf.reset_default_graph()

# We define our inputs
x_ph = tf.placeholder(tf.float32, (None, 5))
y_ph = tf.placeholder(tf.float32, (None))

#We define a linear regression layer
yhat = tf.layers.dense(x_ph, 1, name='regression_coefficients')

#we calculate our loss
loss = tf.reduce_mean(tf.square(yhat[:,0] - y_ph))

#We minimize the loss
optimizer = tf.train.GradientDescentOptimizer(.1).minimize(loss)

#Tensorflow necessary initialization of our model.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

with tf.variable_scope("regression_coefficients", reuse=True):
    weights = tf.get_variable("kernel")

loss_achieved = 0
ROI_achieved = 0
GT = np.array([1,-.5,0,0,0])
#We take optimization steps training the model to fit X, Y
for i in range(4000):
    sess.run(optimizer, {x_ph:X, y_ph:Y})
    lossval = sess.run(loss, {x_ph:X, y_ph:Y})
    if lossval < .011 and not loss_achieved:
        print('Loss Achieved', i)
        loss_achieved = 1
    if i % 1000 == 0:
        coefs = sess.run(weights[:,0]).round(2)
        if (GT - coefs < .1).all() and not ROI_achieved:
            print('ROI Achieved', i)
            ROI_achieved = 1
    if loss_achieved and ROI_achieved:
        break
    
#We analyze the regression coefficients (ROI) the model learned
with tf.variable_scope("regression_coefficients", reuse=True):
    weights = tf.get_variable("kernel")
print('Regression coefficients From Tensorflow:')

coefs = sess.run(weights[:,0]).round(2)
print(list(zip(['x1', 'x2', 'x3', 'x4', 'x5'], coefs)))

In [ ]:
coefs

In [ ]:
i

In [ ]:
lossval
